# Genetic Toggle Switch
***
Gardner et al. Nature (1999) 'Construction of a genetic toggle switch in Escherichia coli'
***
## Setup the Environment
***

In [ ]:
import numpy

MatPlotLib and Plotly are used for creating custom visualizations

In [ ]:
from matplotlib import pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [ ]:
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go

In [ ]:
import gillespy2

***
## Create the Genetic Toggle Switch Model
***

In [ ]:
def create_genetic_toggle_switch(parameter_values=None):
    model = gillespy2.Model(name="Toggle_Switch")
    model.volume = 1

    # Variables
    U = gillespy2.Species(name="U", initial_value=10, mode="discrete")
    V = gillespy2.Species(name="V", initial_value=10, mode="discrete")
    model.add_species([U, V])

    # Parameters
    alpha1 = gillespy2.Parameter(name="alpha1", expression="1")
    alpha2 = gillespy2.Parameter(name="alpha2", expression="1")
    beta = gillespy2.Parameter(name="beta", expression="2")
    gamma = gillespy2.Parameter(name="gamma", expression="2")
    mu = gillespy2.Parameter(name="mu", expression="1")
    model.add_parameter([alpha1, alpha2, beta, gamma, mu])

    # Reactions
    cu = gillespy2.Reaction(
        name="cu",
        reactants={}, products={'U': 1},
        propensity_function="alpha1/(1+pow(V,beta))",
        ode_propensity_function="alpha1/(1+pow(V,beta))"
    )
    cv = gillespy2.Reaction(
        name="cv",
        reactants={}, products={'V': 1},
        propensity_function="alpha2/(1+pow(U,gamma))",
        ode_propensity_function="alpha2/(1+pow(U,gamma))"
    )
    du = gillespy2.Reaction(
        name="du", rate="mu",
        reactants={'U': 1}, products={}
    )
    dv = gillespy2.Reaction(
        name="dv", rate="mu",
        reactants={'V': 1}, products={}
    )
    model.add_reaction([cu, cv, du, dv])

    # Timespan
    tspan = gillespy2.TimeSpan.arange(1, t=101)
    model.timespan(tspan)
    return model

### Instantiate the Model

In [ ]:
model = create_genetic_toggle_switch()

***
## Simulation Parameters
***

In [ ]:
def configure_simulation():
    solver = gillespy2.SSACSolver(model=model)
    kwargs = {
        "solver":solver,
        "number_of_trajectories":20,
        # "seed":None,
        # "tau_tol":0.03,
        # "integrator_options":{'rtol': 0.001, 'atol': 1e-06},
    }
    return kwargs

***
## Post Processing
***
### Feature extraction function
What value(s) do you want to extract from the simulation trajectory

In [ ]:
def population_at_last_timepoint(c, res):
    if c.verbose:
        print(f'population_at_last_timepoint {c.variable_of_interest}={res[c.variable_of_interest][-1]}')
    return res[c.variable_of_interest][-1]

### Aggregation function
How do we combine the values from multiple trajectores

In [ ]:
def average_of_ensemble(c, data):
    a = numpy.average(data)
    if c.verbose:
        print(f'average_of_ensemble = {a}')
    return a

***
## Parameter Sweep
***

In [ ]:
class ParameterSweep2D():

    def run(c, kwargs, verbose=False):
        c.verbose = verbose
        fn = c.feature_extraction
        ag = c.ensemble_aggragator
        data = numpy.zeros((len(c.p1_range), len(c.p2_range)))
        for i, v1 in enumerate(c.p1_range):
            for j, v2 in enumerate(c.p2_range):
                if c.verbose:
                    print(f'running {c.p1}={v1}, {c.p2}={v2}')
                if(c.number_of_trajectories > 1):
                    tmp_results = model.run(**kwargs, variables={c.p1:v1, c.p2:v2})
                    data[i, j] = ag([fn(x) for x in tmp_results])
                else:
                    tmp_result = model.run(**kwargs, variables={c.p1:v1, c.p2:v2})
                    data[i, j] = c.feature_extraction(tmp_result)
        c.data = data


    def plot(c):
        fig, ax = plt.subplots(figsize=(8, 8))
        plt.imshow(c.data)
        ax.set_xticks(numpy.arange(c.data.shape[1]) + 0.5, minor=False)
        ax.set_yticks(numpy.arange(c.data.shape[0]) + 0.5, minor=False)
        plt.title(f'Parameter Sweep - Variable: {c.variable_of_interest}')
        ax.set_xticklabels(c.p1_range, minor=False, rotation=90)
        ax.set_yticklabels(c.p2_range, minor=False)
        ax.set_xlabel(c.p1, fontsize=16, fontweight='bold')
        ax.set_ylabel(c.p2, fontsize=16, fontweight='bold')
        divider = make_axes_locatable(ax)
        cax = divider.append_axes('right', size='5%', pad=0.2)
        _ = plt.colorbar(ax=ax, cax=cax)


    def plotplotly(c, return_plotly_figure=False):
        xaxis_ticks = c.p1_range
        yaxis_ticks = c.p2_range

        trace_list = [go.Heatmap(z=c.data, x=xaxis_ticks, y=yaxis_ticks)]
        title = dict(text=f'<b>Parameter Sweep - Variable: {c.variable_of_interest}</b>', x=0.5)
        xaxis_label = dict(title=f'<b>{c.p1}</b>')
        yaxis_label = dict(title=f'<b>{c.p2}</b>')

        layout = go.Layout(title=title, xaxis=xaxis_label, yaxis=yaxis_label)

        fig = dict(data=trace_list, layout=layout)

        if return_plotly_figure:
            return fig
        iplot(fig)

In [ ]:
class ParameterSweepConfig(ParameterSweep2D):
    # What class defines the GillesPy2 model
    model = create_genetic_toggle_switch()
    # ENTER PARAMETER 1 HERE
    p1 = 'alpha1'
    # ENTER PARAMETER 2 HERE
    p2 = 'alpha2'
    # ENTER START VALUE FOR P1 RANGE HERE
    p1_min = 0.5 * float(eval(model.get_parameter(p1).expression))
    # ENTER END VALUE FOR P1 RANGE HERE
    p1_max = 1.5 * float(eval(model.get_parameter(p1).expression))
    # ENTER THE NUMBER OF STEPS FOR P1 HERE
    p1_steps = 11
    p1_range = numpy.linspace(p1_min, p1_max, p1_steps)
    # ENTER START VALUE FOR P2 RANGE HERE
    p2_min = 0.5 * float(eval(model.get_parameter(p2).expression))
    # ENTER END VALUE FOR P2 RANGE HERE
    p2_max = 1.5 * float(eval(model.get_parameter(p2).expression))
    # ENTER THE NUMBER OF STEPS FOR P2 HERE
    p2_steps = 11
    p2_range = numpy.linspace(p2_min, p2_max, p2_steps)
    # ENTER VARIABLE OF INTEREST HERE
    variable_of_interest = 'U'
    number_of_trajectories = 20
    # What feature of the simulation are we examining
    feature_extraction = population_at_last_timepoint
    # for ensemble resutls: how do we aggreggate the values
    ensemble_aggragator = average_of_ensemble

***
## Run the Parameter Sweep
***

In [ ]:
kwargs = configure_simulation()
ps = ParameterSweepConfig()
%time ps.run(kwargs)

***
## Visualization
***

In [ ]:
ps.plot()

In [ ]:
ps.plotplotly()